In [1]:

import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import seaborn as sns
import keras.applications.xception as xception
import zipfile
import sys
import time
import tensorflow.keras as keras
import tensorflow as tf
import re

from PIL import Image
from keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling2D, Input, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input
from keras.models import Model, Sequential
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.layers import Lambda
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

2023-06-04 20:55:59.236353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
IMAGE_WIDTH = 320    
IMAGE_HEIGHT = 320
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3


# Path where our data is located
base_path = r'/Users/fathinrahma/Downloads/garbage_classification/'

# Dictionary to save our 12 classes
categories = {0: 'paper', 1: 'cardboard', 2: 'plastic', 3: 'metal', 4: 'trash', 5: 'battery',
              6: 'shoes', 7: 'clothes', 8: 'green-glass', 9: 'brown-glass', 10: 'white-glass',
              11: 'biological'}

In [3]:
def add_class_name_prefix(df, col_name):
    df[col_name] = df[col_name].apply(lambda x: x[:re.search("\d",x).start()] + '/' + x)
    return df

# list conatining all the filenames in the dataset
filenames_list = []
# list to store the corresponding category, note that each folder of the dataset has one class of data
categories_list = []

for category in categories:
    filenames = os.listdir(base_path + categories[category])
    
    filenames_list = filenames_list  +filenames
    categories_list = categories_list + [category] * len(filenames)
    
df = pd.DataFrame({
    'filename': filenames_list,
    'category': categories_list
})

df = add_class_name_prefix(df, 'filename')

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

print('number of elements = ' , len(df))

number of elements =  15515


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import keras.applications.xception as xception

xception_layer = xception.Xception(include_top = False, input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
                       weights = 'imagenet')

# We don't want to train the imported weights
xception_layer.trainable = False


model = Sequential()
model.add(keras.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))

#create a custom layer to apply the preprocessing
def xception_preprocessing(img):
  return xception.preprocess_input(img)

model.add(Lambda(xception_preprocessing))

model.add(xception_layer)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(len(categories), activation='softmax')) 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 320, 320, 3)       0         
                                                                 
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 12)                24588     
                                                                 
Total params: 20,886,068
Trainable params: 24,588
Non-trainable params: 20,861,480
_________________________________________________________________


In [5]:
early_stop = EarlyStopping(patience = 2, verbose = 1, monitor='val_categorical_accuracy' , mode='max', min_delta=0.001, restore_best_weights = True)

callbacks = [early_stop]

In [6]:
df["category"] = df["category"].replace(categories) 

# We first split the data into two sets and then split the validate_df to two sets
train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.5, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

print('train size = ', total_validate , 'validate size = ', total_validate, 'test size = ', test_df.shape[0])

train size =  1551 validate size =  1551 test size =  1552


In [7]:
batch_size=64

train_datagen = image.ImageDataGenerator(
    
    ###  Augmentation Start  ###
    
    #rotation_range=30,
    #shear_range=0.1,
    #zoom_range=0.3,
    #horizontal_flip=True,
    #vertical_flip = True,
    #width_shift_range=0.2,
    #height_shift_range=0.2
    
    ##  Augmentation End  ###
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    base_path, 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

validation_datagen = image.ImageDataGenerator()

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    base_path, 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Found 12412 validated image filenames belonging to 12 classes.
Found 1551 validated image filenames belonging to 12 classes.


In [8]:
EPOCHS = 20
history = model.fit_generator(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

/var/folders/g4/c07sgvs57wvfkh2285l6sdrh0000gn/T/com.apple.sharingd/ipykernel_78910/2716048666.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20


2023-06-04 20:56:23.935600: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


193/193 [==============================] - ETA: 0s - loss: 0.6090 - categorical_accuracy: 0.8412 

2023-06-04 23:13:06.643033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


193/193 [==============================] - 9148s 47s/step - loss: 0.6090 - categorical_accuracy: 0.8412 - val_loss: 0.2909 - val_categorical_accuracy: 0.9284
Epoch 2/20
193/193 [==============================] - 7434s 39s/step - loss: 0.2344 - categorical_accuracy: 0.9407 - val_loss: 0.2294 - val_categorical_accuracy: 0.9362
Epoch 3/20
193/193 [==============================] - 5618s 29s/step - loss: 0.1787 - categorical_accuracy: 0.9517 - val_loss: 0.2023 - val_categorical_accuracy: 0.9414
Epoch 4/20
193/193 [==============================] - 6216s 32s/step - loss: 0.1493 - categorical_accuracy: 0.9594 - val_loss: 0.1922 - val_categorical_accuracy: 0.9434
Epoch 5/20
193/193 [==============================] - 5564s 29s/step - loss: 0.1285 - categorical_accuracy: 0.9648 - val_loss: 0.1783 - val_categorical_accuracy: 0.9518
Epoch 6/20
193/193 [==============================] - 6211s 32s/step - loss: 0.1124 - categorical_accuracy: 0.9704 - val_loss: 0.1766 - val_categorical_accuracy: 0.94

In [9]:
model.save(r'/Users/fathinrahma/Downloads/transfer_learning.h5',save_format='h5')

In [10]:
model=tf.keras.models.load_model('/Users/fathinrahma/Downloads/transfer_learning.h5')

In [59]:
keras_file = '/Users/fathinrahma/Downloads/transfer_learning.h5'
tf.keras.models.save_model(model, keras_file)

# Convert to TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
tflite_file = '/Users/fathinrahma/Downloads/transfer_learning.tflite'
with open(tflite_file, 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/g4/c07sgvs57wvfkh2285l6sdrh0000gn/T/com.apple.sharingd/tmpz0sfcnx9/assets


INFO:tensorflow:Assets written to: /var/folders/g4/c07sgvs57wvfkh2285l6sdrh0000gn/T/com.apple.sharingd/tmpz0sfcnx9/assets
2023-06-05 15:50:48.926727: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-05 15:50:48.926757: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-05 15:50:48.927331: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/g4/c07sgvs57wvfkh2285l6sdrh0000gn/T/com.apple.sharingd/tmpz0sfcnx9
2023-06-05 15:50:48.985460: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-05 15:50:48.985490: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/g4/c07sgvs57wvfkh2285l6sdrh0000gn/T/com.apple.sharingd/tmpz0sfcnx9
2023-06-05 15:50:49.230191: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-06-05 15:50:50.308355: I tensor